# with recursive を試す

参考: [link](https://qiita.com/Shoyu_N/items/f1786f99545fa5053b75)

再起 with は、

```sql
with テーブル名 as (
select ...
union all
select ...
  from テーブル名 ...
),
...
```

という形をしている。

- union all で 2 つの select を連結
- 前半は、自分自身に言及しない`非再帰項`
- 後半は、自分自身を参照する`再帰項`

評価プロセスを理解するには、以下の 3 つのテーブルをイメージすると良い。

- with テーブル
- ワークテーブル
- 結果テーブル

評価の進行イメージ：

1. 初期化：非再帰項を評価する → 結果を、with テーブル、結果テーブルに代入
2. 再帰項を評価する → このとき、自分自身への参照は with テーブルを見る。結果をワークテーブルに代入する
3. ワークテーブルが空でなければ、
   - 結果テーブルにワークテーブルを union
   - with テーブルをワークテーブルで置き換え、ワークテーブルを空に
   - 2.に戻る
4. 結果テーブルを返す

- with テーブル: 再帰項が参照するテーブル
- ワークテーブル: 一回の再帰の結果を保存するところ
- 結果テーブル: 最終的な結果を積み上げとくところ

という感じと理解。


とりあえず、連番のテーブルを作ってみる。


In [2]:
import pandas as pd
import sqlite3

def connect():
    con = sqlite3.connect(":memory:")
    con.row_factory = sqlite3.Row
    return con

def query(sql: str) -> pd.DataFrame:
    con = connect()
    return pd.read_sql_query(sql, con)

In [5]:
sql = f"""
with recursive numbers as (
  select 1 as num
    union all
  select
    num + 1 as num
  from
    numbers
  where
    num < 9  
)
select * from numbers
"""
query(sql)

,num
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9


`num < 9`だが、結果には 9 の行も含まれているのに注意。ちょっと見た目的に分かりづらいが、想定の挙動。

`num < 9`という条件だと、with テーブルに 8 が含まれているときには、テーブルが空にならない。このとき、再帰項を評価して得られるのは、それに 1 足した値なので、9 が返ってくる。
次の再帰では、`num < 9`という条件によって結果が空になり再帰が止まる。
